In [14]:
import os
filename = '/usr/local/Cellar/apache-spark/2.2.1/libexec/python/pyspark/shell.py'
exec(open(filename).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.2.1
      /_/

Using Python version 3.6.1 (default, May 11 2017 13:04:09)
SparkSession available as 'spark'.


In [15]:
from pyspark.sql import SparkSession

In [16]:
# May take awhile locally
spark = SparkSession.builder.appName("Exploration").getOrCreate()

## First, use a small dataset to see some usage of DataFrame in Spark

In [4]:
# Let Spark know about the header and infer the Schema types!
df = spark.read.csv('../data/play/20170301_play.log.fn', sep = '\t', inferSchema=True, header=True)

In [17]:
df.printSchema()

root
 |-- uid: double (nullable = true)
 |-- device: string (nullable = true)
 |-- song_id: string (nullable = true)
 |-- song_type: string (nullable = true)
 |-- song_name: string (nullable = true)
 |-- singers: string (nullable = true)
 |-- play_time: string (nullable = true)
 |-- song_length: string (nullable = true)
 |-- paid_flag: string (nullable = true)
 |-- file_name: string (nullable = true)



In [18]:
df.show()

+------------+------+---------+---------+--------------------+--------------------+---------+-----------+---------+------------------+
|         uid|device|  song_id|song_type|           song_name|             singers|play_time|song_length|paid_flag|         file_name|
+------------+------+---------+---------+--------------------+--------------------+---------+-----------+---------+------------------+
|1.54422682E8|   ar |20870993 |       1 |                 用情 |              狮子合唱团 |   22013 |       332 |       0 | 20170301_play.log|
|1.54421907E8|   ip | 6560858 |       0 |             表情不要悲伤 |    伯贤&D.O.&张艺兴&朴灿烈 |      96 |       161 |       0 | 20170301_play.log|
| 1.5442263E8|   ar | 3385963 |       1 |Baby, Don't Cry(人...|                EXO |  235868 |       235 |       0 | 20170301_play.log|
|1.54410267E8|   ar | 6777172 |       0 |   3D-环绕音律1(3D Mix) |             McTaiM |     164 |       237 |       0 | 20170301_play.log|
|1.54407793E8|   ar |19472465 |       0 |              

In [7]:
df.describe()

DataFrame[summary: string, uid: string, device: string, song_id: string, song_type: string, song_name: string, singers: string, play_time: string, song_length: string, paid_flag: string, file_name: string]

In [8]:
# total number of samples
df.count()

3422934

In [9]:
# the number of unique id in the dataset
df.select('uid').distinct().count()

175192

## Next, use the whole dataset to transform to DataFrame in Spark

In [10]:
# Let Spark know about the header and infer the Schema types!
df_whole = spark.read.csv('../data/all_play.log.fn', sep = '\t', inferSchema=True, header=True)

In [11]:
df_whole.printSchema()

root
 |-- uid: string (nullable = true)
 |-- device: string (nullable = true)
 |-- song_id: string (nullable = true)
 |-- song_type: string (nullable = true)
 |-- song_name: string (nullable = true)
 |-- singers: string (nullable = true)
 |-- play_time: string (nullable = true)
 |-- song_length: string (nullable = true)
 |-- paid_flag: string (nullable = true)
 |-- file_name: string (nullable = true)



In [12]:
# total number of samples in the whole dataset
df_whole.count()

164667143

In [13]:
# the number of unique id in the whole dataset
df_whole.select('uid').distinct().count()

871702

## Finally, load the uid list(a text file)

In [49]:
# Load data from txt file without header!
import pandas as pd
schema = ['uid', 'user_name']
raw_uid_list = pd.read_csv('../data/all_uid.txt',delimiter='     ',header=None,index_col=None,names=schema,engine='python')

In [50]:
raw_uid_list.head(5)

,uid,user_name
0,154464765,004402142270997
1,154788607,00:08:22:92:17:0a
2,154659978,00:08:22:98:e1:98
3,154681746,00:08:22:ba:74:07
4,154391982,00:08:22:fc:3d:ca


In [55]:
# transfer the uid list to set
uid_set = set(raw_uid_list.uid.unique())

In [58]:
# the number of uid in the uid_list
len(uid_set)

264715

### Difference of user numbers between all_uid.txt and all_play.log.fn !

The distinct number of user id in all_uid.txt: 264715

The distinct number of user id in all_play.log.fn: 871702